In [16]:
import numpy as np
import tensorflow as tf
from pprint import pprint

In [38]:
from tensorflow.keras import layers, regularizers, initializers, activations, models
import tensorflow.keras.backend as K

In [25]:
def mish(x):
    return x*activations.tanh(K.softplus(x))

In [99]:
def conv(x, filters, kernel_size, downsampling=False, activation='leaky', batch_norm=True):
    if downsampling:
        x = layers.ZeroPadding2D(padding=((1, 0), (1, 0)))(x) # top & left padding
        padding = 'valid'
        strides = 2
    else:
        padding = 'same'
        strides = 1
    x = layers.Conv2D(filters, 
                      kernel_size, 
                      strides=strides, 
                      padding=padding, 
                      use_bias=not batch_norm, 
                      kernel_regularizer=regularizers.l2(0.0005),
                      kernel_initializer=initializers.RandomNormal(stddev=0.01), 
                      bias_initializer=initializers.Zeros())(x)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    if activation == 'mish':
        x = mish(x)
    elif activation == 'leaky':
        x = layers.LeakyReLU(alpha=0.1)(x)
    return x

'''
filters1: 1x1, 
filters2: 3x3
'''
def residual_block(x, filters1, filters2, activation='leaky'):
    y = conv(x, filters1, kernel_size=1, activation=activation)
    y = conv(y, filters2, kernel_size=3, activation=activation)
    return layers.Add()([x, y])

'''Cross Stage Partial Network (CSPNet)
transition_bottleneck_dims: 1x1 bottleneck
output_dims: 3x3

'''

# def csp_block(x, residual_bottleneck_dims, residual_out, repeat):
def csp_block(x, residual_out, repeat, residual_bottleneck=False):
    route = x
    route = conv(route, residual_out, 1, activation="mish")
    x = conv(x, residual_out, 1, activation="mish")
    for i in range(repeat):
        x = residual_block(x, 
                           residual_out // 2 if residual_bottleneck else residual_out, 
                           residual_out, 
                           activation="mish")
    x = conv(x, residual_out, 1, activation="mish")

    x = layers.Concatenate()([x, route])
    return x

def darknet53(x):
    x = conv(x, 32, 3)
    x = conv(x, 64, 3, downsampling=True)

    for i in range(1):
        x = residual_block(x, 32, 64)
    x = conv(x, 128, 3, downsampling=True)

    for i in range(2):
        x = residual_block(x, 64, 128)
    x = conv(x, 256, 3, downsampling=True)

    for i in range(8):
        x = residual_block(x, 128, 256)
    route_1 = x
    x = conv(x, 512, 3, downsampling=True)

    for i in range(8):
        x = residual_block(x, 256, 512)
    route_2 = x
    x = conv(x, 1024, 3, downsampling=True)

    for i in range(4):
        x = residual_block(x, 512, 1024)

    return route_1, route_2, x

def cspdarknet53(x):
    x = conv(x, 32, 3)
    x = conv(x, 64, 3, downsampling=True)
    
    x = csp_block(x, residual_out=64, repeat=1, residual_bottleneck=True)
    x = conv(x, 64, 1, activation='mish')
    x = conv(x, 128, 3, activation='mish', downsampling=True)
    
    x = csp_block(x, residual_out=64, repeat=2)
    x = conv(x, 128, 1, activation='mish')
    x = conv(x, 256, 3, activation='mish', downsampling=True)
    
    x = csp_block(x, residual_out=128, repeat=8)
    x = conv(x, 256, 1, activation='mish')
    route1 = x
    x = conv(x, 512, 3, activation='mish', downsampling=True)

    x = csp_block(x, residual_out=256, repeat=8)
    x = conv(x, 512, 1, activation='mish')
    route2 = x
    x = conv(x, 1024, 3, activation='mish', downsampling=True)
    
    x = csp_block(x, residual_out=512, repeat=4)

    x = conv(x, 1024, 1, activation="mish")
    x = conv(x, 512, 1)
    x = conv(x, 1024, 3)
    x = conv(x, 512, 1)
    
    x = layers.Concatenate()([
        layers.MaxPooling2D(pool_size=13, strides=1, padding='same')(x),
        layers.MaxPooling2D(pool_size=9, strides=1, padding='same')(x),
        layers.MaxPooling2D(pool_size=5, strides=1, padding='same')(x),
        x
    ])
    x = conv(x, 512, 1)
    x = conv(x, 1024, 3)
    x = conv(x, 512, 1)
    return route1, route2, x

In [39]:
input = layers.Input((128, 128, 3))
output = darknet53(input)
model1 = models.Model(input, output)

In [44]:
model1.outputs

[<tf.Tensor 'add_75/add:0' shape=(?, 16, 16, 256) dtype=float32>,
 <tf.Tensor 'add_83/add:0' shape=(?, 8, 8, 512) dtype=float32>,
 <tf.Tensor 'add_87/add:0' shape=(?, 4, 4, 1024) dtype=float32>]

In [46]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_152 (Conv2D)             (None, 128, 128, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_151 (BatchN (None, 128, 128, 32) 128         conv2d_152[0][0]                 
__________________________________________________________________________________________________
zero_padding2d_29 (ZeroPadding2 (None, 129, 129, 32) 0           batch_normalization_151[0][0]    
______________________________________________________________________________________________

In [100]:
input = layers.Input((128, 128, 3))
output = cspdarknet53(input)
model2 = models.Model(input, output)

In [101]:
model2.output

(<tf.Tensor 'mul_697:0' shape=(?, 16, 16, 256) dtype=float32>,
 <tf.Tensor 'mul_718:0' shape=(?, 8, 8, 512) dtype=float32>,
 <tf.Tensor 'leaky_re_lu_260/LeakyRelu:0' shape=(?, 4, 4, 512) dtype=float32>)

In [102]:
model2.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_942 (Conv2D)             (None, 128, 128, 32) 864         input_21[0][0]                   
__________________________________________________________________________________________________
batch_normalization_941 (BatchN (None, 128, 128, 32) 128         conv2d_942[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_253 (LeakyReLU)     (None, 128, 128, 32) 0           batch_normalization_941[0][0]    
___________________________________________________________________________________________

In [94]:
# tf.keras.utils.plot_model(model2, show_shapes=True, show_layer_names=True)

In [27]:
# x = tf.reshape(tf.constant([[0, 1, 2], [3, 4, 5], [7, 8, 9]]), ((1, 3, 3, 1)))
# x = layers.ZeroPadding2D(padding=((1, 0), (1, 0)))(x)
# pprint(tf.keras.backend.eval(x).reshape(4, 4))

array([[0, 0, 0, 0],
       [0, 0, 1, 2],
       [0, 3, 4, 5],
       [0, 7, 8, 9]], dtype=int32)
